### 자연어처리
- 사람이 사용하는 글,언어,음성 등을 분석하고 예측하는 분야

In [2]:
# 원하는 task를 쉽게 불러서 사용할 수 있도록 지원
from transformers import pipeline

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
DLL load failed while importing _imaging: 지정된 모듈을 찾을 수 없습니다.

In [ ]:
# 감정분석
sentiment_model = pipeline("sentiment-analysis")

In [ ]:
sentiment_model(["I love you","I hate you"])

In [ ]:
fill_mask_model = pipeline("fill-mask", model='xlm-roberta-base')

In [ ]:
fill_mask_model("I want to <mask> home")

In [ ]:
# 문장생성
text_generation_model = pipeline("text-generation")

In [ ]:
text_generation_model("I just want to")

In [ ]:
# 번역모델(한글->영어)
translation_model = pipeline("translation_ko_to_en")

In [ ]:
translation_model("안녕하세요")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

text = """
주인공 강인구(하정우)는 ‘수리남에서 홍어가 많이 나는데 다 갖다버린다’는 친구 
박응수(현봉식)의 얘기를 듣고 수리남산 홍어를 한국에 수출하기 위해 수리남으로 간다. 
국립수산과학원 측은 “실제로 남대서양에 홍어가 많이 살고 아르헨티나를 비롯한 남미 국가에서 홍어가 많이 잡힌다”며 
“수리남 연안에도 홍어가 많이 서식할 것”이라고 설명했다.

그러나 관세청에 따르면 한국에 수리남산 홍어가 수입된 적은 없다. 
일각에선 “돈을 벌기 위해 수리남산 홍어를 구하러 간 설정은 개연성이 떨어진다”는 지적도 한다. 
드라마 배경이 된 2008~2010년에는 이미 국내에 아르헨티나, 칠레, 미국 등 아메리카산 홍어가 수입되고 있었기 때문이다. 
실제 조봉행 체포 작전에 협조했던 ‘협력자 K씨’도 홍어 사업이 아니라 수리남에 선박용 특수용접봉을 파는 사업을 하러 수리남에 갔었다.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=100)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

In [ ]:
import torch
x= torch.rand(5,3)
print(x)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from melo.api import TTS

# Speed is adjustable
speed = 1.0
device = 'cpu' # or cuda:0

text = "안녕하세요! 오늘은 날씨가 정말 좋네요."
model = TTS(language='KR', device=device)
speaker_ids = model.hps.data.spk2id

output_path = 'kr.wav'
model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)

In [ ]:
from gtts import gTTS

text = "안녕하세요! 오늘은 날씨가 정말 좋네요."
output_path = 'kr.wav'

# Create gTTS object
tts = gTTS(text=text, lang='ko')

# Save to a file
tts.save(output_path)

print(f"Saved the TTS output to {output_path}")

In [ ]:
# 객체탐지
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes


# print results
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )
